# 🔍 Exploring Regular Expression Engine Implementations 🚀

I want to understand what goes on inside a regex engine by writing a *simplified, illustrative* model that does **not** represent a fully compliant engine, but it will be more than sufficient for me to understand the core.

**1. 🧱 Regex Building Blocks: A Quick Recap**

   *   Metacharacters (`.`, `\w`, `\d`, `\s`): Special symbols that define patterns.
   *   Quantifiers (`*`, `+`, `?`, `{n,m}`): Control repetition.
   *   Character Classes (`[a-z]`, `[0-9]`, `[^abc]`): Define sets of characters.
   *   Grouping (`()`): Create sub-patterns.
   *   Anchors (`^ $`): (Start & End)
   *   Alternation (`|`): (OR: `a|b` means "a OR b")

**2. ⚙️ Regex to NFA: Representing a Regexe as NFA**

   This section dives into the conversion of regular expressions into Non-deterministic Finite Automata (NFAs). An NFA is a state machine that accepts or rejects strings based on defined transitions. Converting a regex to an NFA provides a visual and computational representation of the pattern.

   *   **What is an NFA?** An NFA consists of:
         *   A set of states $Q$
         *   An input alphabet $\Sigma$
         *   A transition function $\delta: Q \times (\Sigma \cup \{\epsilon\}) \rightarrow \mathcal{P}(Q)$  (where $\mathcal{P}(Q)$ is the power set of Q, meaning it returns a *set* of possible next states, and $\epsilon$ represents the empty string/epsilon transition)
         *   A start state $q_0 \in Q$
         *   A set of accept states $F \subseteq Q$


   *   **Thompson's Construction:** A common algorithm for converting a regex to an NFA. It builds the NFA step-by-step based on the regex operators.  We'll implement a similar approach.

   *   **Example:** The regex `a*` can be represented by an NFA with states $q_0$ and $q_1$, where $q_0$ is the start state and $q_1$ is the accept state. There's an 'a' transition *looping* on $q_1$ and an epsilon transition from $q_0$ to $q_1$.


**2. 🚂 Shunting Yard Algorithm: From Infix to Postfix**

   We'll implement the Shunting Yard algorithm to transform familiar infix regex (like `a+b*c`) into postfix (Reverse Polish Notation - RPN), which is easier for machines to evaluate.

   **Example:** `a + b * c  -->  a b c * +`



## **I. NFA Representation**

In [14]:
# How to represent a NFA maybe as
# 2d array representing transitions next_state = transition_table[row][col];
# events are rows & states are columns
# how to match abc, this is how the transition table should look like
# last state is always accept state?? maybe
#   s0 s1 s2 s3
# a 1  N  N
# b N  2  N
# c N  N  3
from typing import List, Tuple, Union


EPSILON_TRANSITION = None
EPSILON = 'ε'
class State:
    def __init__(self, size, name, is_accept, is_epsilon=False):
        self.transitions = [0] * size  # Initialize the transitions list with 0 representing a failure state
        self.name = name
        self.is_accept = is_accept
        self.is_epsilon = is_epsilon
        
    def set_transition_vec(self, range: range, nextStateIdx: int) -> None:
        for i in range:
            self.transitions[i] = nextStateIdx 
    def __repr__(self):
        return f"State(name='{self.name}', is_accept={self.is_accept}, transitions={self.transitions})"
        
class EngineNFA:
    def __init__(self, alphabet):
        self.transitions_table = [] 
        self.alphabet = alphabet.copy() # the language lookup this NFA can describe {char: int}
        self.alphabet[EPSILON] = max(alphabet.values()) + 1 # EPSILON is always present in the alphabet
        
    def add_state(self, name, transition: range, nextStatesIdx: Tuple[int, ...], is_accept=False, is_epsilon=False) -> None:
        new_state = State(len(self.alphabet.keys()), name, is_accept, is_epsilon)  # Create a new instance of State
        new_state.set_transition_vec(transition, nextStatesIdx)
        self.transitions_table.append(new_state)  # Add state to transitions table
            
    def remove_accept_state(self):
        #By convension the last state is always an accept state.
        del self.transitions_table[-1]
        
    def increment_state_by(self, state: State, inc: int):
        state.name = self.get_incremented_state_name(state, inc)
        state.transitions = [(x[0] + inc, *x[1:]) if isinstance(x, tuple)
                                and len(x) > 0 and x[0] != 0 else x for x in state.transitions]
        return state
    
    def insert_state(self, state: State):
        self.transitions_table.append(state)   
                
    def get_incremented_state_name(self, state: State, inc: int):
        num_as_str = ""
        i = 0
        while i < len(state.name):
            if state.name[i] == 'S':
                i += 1  # Move past the 'S'
                while i < len(state.name) and state.name[i].isdigit():
                    num_as_str += state.name[i]
                    i += 1
            else:
                i += 1
        return f"S{int(num_as_str) + inc}"

    def is_match(self, input: Union[str, List[str]]) -> bool:
        """
        TODO: maybe explore a non backtracking implementation
        Non-backtracking implementation: A NFA machine can be considered as being at multiple states at the same time
        Implements an iterative backtracking solution to test whether a regex matches a string.
        """
        stack = []
        # push current char index and current state (int, State)
        stack.append((0, self.transitions_table[0]))
        while len(stack) > 0:
            idx, curr_state = stack.pop()
            if(curr_state.is_accept):
                return True
            # input string consumed entirely and not end up in an accept state
            if(idx >= len(input)):
                return False
            char = input[idx]
            if(char not in self.alphabet):
                raise ValueError(f"Invalid character: {char} is not a valid character in the defined alphabet")
            transitions = curr_state.transitions[self.alphabet[char]] # int or tuple
            
            if isinstance(transitions, int):
                transitions = tuple([transitions])
                
            # Possible transitions are pushed in reverse order
            for s in reversed(transitions):
                if s != 0:
                    next_char_idx = idx if curr_state.is_epsilon else idx + 1 
                    stack.append((next_char_idx, self.transitions_table[s]))
        return False
    
    def dump(self):
        # Find the maximum string length of any element (int or tuple) for elemnts alignment
        max_len = 0
        for row in self.transitions_table:
            for element in row.transitions:
                element_str = str(element)
                max_len = max(max_len, len(element_str))
                
        print( ' ' * 5, end=" ")
        for i in range(len(self.transitions_table)):
            padding = ' ' * (max_len - len(str(i)))
            print(str(i) + padding, end=" ")
        print('\n')
        
        for symbol in self.alphabet.keys():
            print(f"{symbol} => ", end=" ")
            for state in self.transitions_table:
                element = str(state.transitions[self.alphabet[symbol]])
                padding = ' ' * (max_len - len(element))
                print(element + padding, end=" ")
            print('\n')
    

In [75]:
# example of a NFA that recognizes the string "acb"
alphabet = {chr(i): i - ord('a') for i in range(ord('a'), ord('c') + 1)}

events = ['a', 'c', EPSILON, EPSILON, 'b']
NFA = EngineNFA(alphabet)
for idx, e in enumerate(events):
    NFA.add_state(f"S{idx}", range(NFA.alphabet[e], NFA.alphabet[e] + 1), (len(
        NFA.transitions_table) + 1,), is_accept=idx == len(events) - 1, is_epsilon=e == EPSILON)

NFA.dump()
print(NFA.is_match(['a', 'c', EPSILON, EPSILON, 'b']))

      0    1    2    3    4    

a =>  (1,) 0    0    0    0    

b =>  0    0    0    0    (5,) 

c =>  0    (2,) 0    0    0    

ε =>  0    0    (3,) (4,) 0    

True


## **II. Regex 2 NFA**


## Step 1: Parsing
#### Actualy, instead of creating a parser for the regex, I will use the _Shunting-Yard Algorithm_ to translate the regex to reverse polish notation.

In [2]:

# Shunting-Yard Algorithm
#  Operator precedance and associativity, table taken from unix: (Not all operators are supported in below implementation.)
#  +---+----------------------------------------------------------+
#  |   |             ERE Precedence (from high to low)            |
#  +---+----------------------------------------------------------+
#  | 1 | Collation-related bracket symbols | [==] [::] [..]       |
#  | 2 | Escaped characters                | \<special character> |
#  | 3 | Bracket expression                | []                   |
#  | 4 | Grouping                          | ()                   |
#  | 5 | Single-character-ERE duplication  | * + ? {m,n}          |
#  | 6 | Concatenation                     | #                    |
#  | 7 | Anchoring                         | ^ $                  |
#  | 8 | Alternation                       | |                    |
#  +---+-----------------------------------+----------------------+
def getPresedence(op):
    opPresedence = {"(": 1, "|": 2, "#": 3, "?":6, "*":6, "+": 6, "^": 5, "$": 5}    
    if op in opPresedence:
        return opPresedence[op]
    else:
        return max(opPresedence.values()) + 1
    
def implicitConcat(regex):
    output = regex[0]
    i = 1
    while(i < len(regex)):
        match regex[i]:
            case char if char in (")", "+", "*", "|", "?"):
                output += regex[i]
            case _:
                if(regex[i-1] == "(" or regex[i-1] == "|"):
                    output += regex[i]
                else:
                    output += "#" + regex[i]
        i +=1
    return output

def plus_to_star(regex):
    i = 1
    output = regex[0]
    while(i < len(regex)):
        match regex[i]:
            case "+":
                prev_symbol = regex[i - 1]
                if(prev_symbol == ')'):
                    end = i - 1
                    start = i - 1
                    while (prev_symbol != "("):
                        start -= 1
                        prev_symbol = regex[start]
                    output += regex[start: end + 1] + "*"
                else:
                    output += prev_symbol + "*"
            case _:
                output += regex[i]
        i +=1
    return output
        
    
def infix2Postfix(regex):
    stack = []
    output = ""
    for char in regex:
        match char:
            case "(":
                stack.append(char)
            case ")":
                while(len(stack) and stack[-1] != '('):
                    output += stack.pop()
                stack.pop()
            case _:
                while(len(stack)):
                    if(getPresedence(stack[-1]) >= getPresedence(char)):
                        output += stack.pop()
                    else: break
                stack.append(char)
                    
    while (len(stack)):
            if (stack[-1] == '(' or stack[-1] == ')'):
                raise ValueError(f"Invalid Expression: Open parenthesis without closing")
            output += stack.pop()
    return output
def pre_process_regex(regex: str) -> str:
    return infix2Postfix(implicitConcat(plus_to_star(regex)))

print(infix2Postfix(implicitConcat("a?b")))
print(plus_to_star("(aaba)+"))
# +------------+---------------+
# | INFIX      | POSTFIX       |
# +------------+---------------+
# | ^xyz$      |  ^x#y#z#$#    |
# | (a|b)+a?|c |  ab|+a#?#c|   |
# | (aa)∣(bb)∣(cc)    | aa#∣#b#b#∣#c#c# |
# +------------+---------------+


a?b#
(aaba)(aaba)*


## Thompson's Construction:

### 1. Union Construction:

#### I. NFA for the single character 'a':

*   State 1 (Start): --a--> State 2 (Accept)
*   NFA1 = {States: {1, 2}, Alphabet: {a}, Start State: 1, Accept State: 2, Transitions: {(1, 'a') -> 2}}

|       | 1   | 2   |
|-------|-----|-----|
| a     | {2} | {}  |


#### II. NFA for the single character 'b':

*   State 3 (Start): --b--> State 4 (Accept)
*   NFA2 = {States: {3, 4}, Alphabet: {b}, Start State: 3, Accept State: 4, Transitions: {(3, 'b') -> 4}}

|       | 3   | 4   |
|-------|-----|-----|
| b     | {4} | {}  |


#### II. NFA for 'a|b' (Union):

*   New Start State: State 0
*   New Accept State: State 5
*   ε-transitions:
    *   State 0 to State 1
    *   State 0 to State 3
    *   State 2 to State 5
    *   State 4 to State 5
##### NFA Properties:
*   **States:** `{0, 1, 2, 3, 4, 5}`
*   **Alphabet:** `{a, b, ε}`
*   **Start State:** `0`
*   **Accept State:** `5`
*   **Transitions:**
    *   `(0, ε) -> 1`
    *   `(0, ε) -> 3`
    *   `(1, a) -> 2`
    *   `(3, b) -> 4`
    *   `(2, ε) -> 5`
    *   `(4, ε) -> 5`

|       | 0     | 1     | 2     | 3     | 4     | 5     |
|-------|-------|-------|-------|-------|-------|-------|
| a     | {}    | {2}   | {}    | {}    | {}    | {}    |
| b     | {}    | {}    | {}    | {4}   | {}    | {}    |
| ε     | {1, 3} | {}    | {5}   | {}    | {5}   | {}   |


### 2. Star Construction:

#### 1. NFA for 'a':

*   State 1 (Start): --a--> State 2 (Accept)

#### 2. NFA for 'a*':

*   Create new Start State: State 0
*   Create new Accept State: State 3

*   Add ε-transition from State 0 to State 1.
*   Add ε-transition from State 0 to State 3.
*   Add ε-transition from State 2 to State 1.
*   Add ε-transition from State 2 to State 3.

##### NFA Properties:

*   **States:** `{0, 1, 2, 3}`
*   **Alphabet:** `{a, ε}`
*   **Start State:** `0`
*   **Accept State:** `3`
*   **Transitions:**
    *   `(0, ε) -> 1`
    *   `(0, ε) -> 3`
    *   `(1, a) -> 2`
    *   `(2, ε) -> 1`
    *   `(2, ε) -> 3`

|       | 0   | 1   | 2   | 3   |
|-------|-----|-----|-----|-----|
| a     | {}  | {2} | {}  | {}  |
| ε     | {1,3} | {}  | {1,3} | {}  |




## Step 2: Building the NFA

In [15]:
class NFABuilder:
    def __init__(self, alphabet, postfix_regex):
        """
        Initializes the NFABuilder object.
        """
        self.postfix_regex = postfix_regex      
        self.alphabet = alphabet
        
    def _atomic(self,events):
        NFA = EngineNFA(self.alphabet)
        for idx, e in enumerate(events):
            NFA.add_state(f"S{idx}", range(NFA.alphabet[e], NFA.alphabet[e] + 1), (len(
                NFA.transitions_table) + 1,), False, e == EPSILON)
        # explicitly add a final accept state
        NFA.add_state(f"S_final", range(0), 0, True, False)
        return NFA
    
    def _concatination(self, nfa1: EngineNFA, nfa2: EngineNFA) -> EngineNFA:
        nfa2.remove_accept_state()
        for i in range(len(nfa1.transitions_table)):
            if not nfa1.transitions_table[i].is_accept:
                new_state = nfa2.increment_state_by(nfa1.transitions_table[i], len(nfa2.transitions_table) + i)
                nfa2.insert_state(new_state)
        # explicitly add a final accept state
        nfa2.add_state(f"S_final", range(0), 0, True, False)
        return nfa2
    
    def _alternate(self, nfa1: EngineNFA, nfa2: EngineNFA) -> EngineNFA:
        union_nfa = EngineNFA(self.alphabet)
        final_state_idx = len(nfa1.transitions_table) + len(nfa2.transitions_table) + 1
        expsilon_idx = union_nfa.alphabet[EPSILON]
        nfa1_start_idx, nfa2_start_idx = len(nfa2.transitions_table) + 1, 1
        
        # add epsilon transition to the starting point of each individual NFA
        union_nfa.add_state(f"S{0}", range(expsilon_idx, expsilon_idx + 1), 
                            (nfa2_start_idx, nfa1_start_idx), False, True)
        
        # the states for nfa1 and nfa2 are the same but the indexes are incremented since we
        # added new states
        for i in range(len(nfa2.transitions_table)):
            if not nfa2.transitions_table[i].is_accept:
                new_state = union_nfa.increment_state_by(nfa2.transitions_table[i],1)
                union_nfa.insert_state(new_state)
            
        union_nfa.add_state(f"S{len(union_nfa.transitions_table)}", range(expsilon_idx, expsilon_idx + 1), 
                            (final_state_idx,), False, True)
        
        for i in range(len(nfa1.transitions_table)):
            if not nfa1.transitions_table[i].is_accept:
                new_state = union_nfa.increment_state_by(nfa1.transitions_table[i], nfa1_start_idx)
                union_nfa.insert_state(new_state)
                
        union_nfa.add_state(f"S{len(union_nfa.transitions_table)}", range(expsilon_idx, expsilon_idx + 1), 
                            (final_state_idx,), False, True)
            
        # explicitly add a final accept state
        union_nfa.add_state(f"S_final", range(0), 0, True, False)
        return union_nfa
    
    def _star(self, nfa: EngineNFA) -> EngineNFA:
        star_nfa = EngineNFA(self.alphabet)
        expsilon_idx = star_nfa.alphabet[EPSILON]
        star_nfa.add_state(f"S{0}", range(expsilon_idx, expsilon_idx + 1), 
                            (1,len(nfa.transitions_table) + 1), False, True)
        for i in range(len(nfa.transitions_table)):
            if not nfa.transitions_table[i].is_accept:
                new_state = star_nfa.increment_state_by(nfa.transitions_table[i],1)
                star_nfa.insert_state(new_state)
            else:
                star_nfa.add_state(f"S{i}", range(expsilon_idx, expsilon_idx + 1), 
                            (1,len(nfa.transitions_table) + 1), False, True)
        # explicitly add a final accept state
        star_nfa.add_state(f"S_final", range(0), 0, True, False)
        return star_nfa
    
    def build_nfa(self):
        """
        Convert a single regex expression in postfix notation to an NFA.
        """
        nfas = []
        for char in self.postfix_regex:
            match char:
                # Single char
                case char if char.isalnum():
                    char_nfa = self._atomic([char])
                    nfas.append(char_nfa)
                    
                # concatination
                case '#':
                    if(len(nfas) < 2):
                        raise ValueError("Append symbol must be preceded with atleast 2 symbols")
                    nfa1 = nfas.pop()
                    nfa2 = nfas.pop()
                    nfas.append(self._concatination(nfa1, nfa2))
                             
                # alternation
                case '|': 
                    if(len(nfas) < 2):
                        raise ValueError("Alternation symbol must be preceded with atleast 2 symbols")
                    nfa1, nfa2 = nfas.pop(), nfas.pop()
                    nfas.append(self._alternate(nfa1, nfa2))
                    
                # Kleene star
                case '*': 
                    nfa = nfas.pop()
                    nfas.append(self._star(nfa))
                    
                # optional
                # a? means match a OR nothing
                # so we can construct it from alternation
                case '?':
                    nfa = nfas.pop()
                    nothing_nfa = self._atomic([EPSILON])
                    nfas.append(self._alternate(nfa, nothing_nfa))

                case _:
                    raise ValueError(f"Unkown operator: {char}")
        return nfas[0]
                                        
alphabet = {chr(i): i - ord('a') for i in range(ord('a'), ord('b') + 1)}
postfix_regex = 'aa#'
builder = NFABuilder(alphabet, postfix_regex)
nfa = builder.build_nfa()
print(nfa.is_match("a"))
# for state in nfa.transitions_table:
#     print(state)
# nfa.dump()


False
